### Assignment 1

We start with the PLR part

In [9]:
%env PYTHONWARNINGS=ignore

from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.simplefilter(action='ignore', category=ConvergenceWarning)
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')
warnings.filterwarnings(
    "ignore",
    message=".*did not converge.*",
    category=ConvergenceWarning
)
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LassoCV, LinearRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.base import TransformerMixin, BaseEstimator, clone
from formulaic import Formula
from flaml.automl import AutoML
np.random.seed(1234)
# set random seed for all other libraries
import random
random.seed(1234)
import os
os.environ['PYTHONHASHSEED'] = '1234'


file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/401k.csv"
data = pd.read_csv(file)
y = data['net_tfa'].values
D = data['e401'].values
D2 = data['p401'].values
D3 = data['a401'].values
X = data.drop(['e401', 'p401', 'a401', 'tw', 'tfa', 'net_tfa', 'tfa_he',
               'hval', 'hmort', 'hequity',
               'nifa', 'net_nifa', 'net_n401', 'ira',
               'dum91', 'icat', 'ecat', 'zhat',
               'i1', 'i2', 'i3', 'i4', 'i5', 'i6', 'i7',
               'a1', 'a2', 'a3', 'a4', 'a5'], axis=1)

class FormulaTransformer(TransformerMixin, BaseEstimator):

    def __init__(self, formula, array=False):
        self.formula = formula
        self.array = array

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        df = Formula(self.formula).get_model_matrix(X)
        if self.array:
            return df.values
        return df
transformer = FormulaTransformer("0 + poly(age, degree=6, raw=True) + poly(inc, degree=8, raw=True) "
                                 "+ poly(educ, degree=4, raw=True) + poly(fsize, degree=2, raw=True) "
                                 "+ male + marr + twoearn + db + pira + hown", array=True)

def dml(X, D, y, modely, modeld, *, nfolds, classifier=False):
    '''
    DML for the Partially Linear Model setting with cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely: the ML model for predicting the outcome y
    modeld: the ML model for predicting the treatment D
    nfolds: the number of folds in cross-fitting
    classifier: bool, whether the modeld is a classifier or a regressor

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the treatment D
    resy: the outcome residuals
    resD: the treatment residuals
    epsilon: the final residual-on-residual OLS regression residual
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)  # shuffled k-folds
    yhat = cross_val_predict(modely, X, y, cv=cv, n_jobs=-1)  # out-of-fold predictions for y
    # out-of-fold predictions for D
    # use predict or predict_proba dependent on classifier or regressor for D
    if classifier:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
    else:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1)
    # calculate outcome and treatment residuals
    resy = y - yhat
    resD = D - Dhat

    # final stage ols based point estimate and standard error
    point = np.mean(resy * resD) / np.mean(resD**2)
    epsilon = resy - point * resD
    var = np.mean(epsilon**2 * resD**2) / np.mean(resD**2)**2
    stderr = np.sqrt(var / X.shape[0])

    return point, stderr, yhat, Dhat, resy, resD, epsilon

def summary(point, stderr, yhat, Dhat, resy, resD, epsilon, X, D, y, *, name):
    '''
    Convenience summary function that takes the results of the DML function
    and summarizes several estimation quantities and performance metrics.
    '''
    return pd.DataFrame({'estimate': point,  # point estimate
                         'stderr': stderr,  # standard error
                         'lower': point - 1.96 * stderr,  # lower end of 95% confidence interval
                         'upper': point + 1.96 * stderr,  # upper end of 95% confidence interval
                         'rmse y': np.sqrt(np.mean(resy**2)),  # RMSE of model that predicts outcome y
                         'rmse D': np.sqrt(np.mean(resD**2)),  # RMSE of model that predicts treatment D
                         'accuracy D': np.mean(np.abs(resD) < .5),  # binary classification accuracy of model for D
                         }, index=[name])
# double lasso with cross-fitting
cv = KFold(n_splits=5, shuffle=True, random_state=123)
lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
lassod = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
result = dml(X, D, y, lassoy, lassod, nfolds=5)
table = summary(*result, X, D, y, name='double lasso')

# penalized logreg for D (default is l2 penalty)
cv = KFold(n_splits=5, shuffle=True, random_state=123)
lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
lgrd = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
result = dml(X, D, y, lassoy, lgrd, nfolds=5, classifier=True)
table = pd.concat([table, summary(*result, X, D, y, name='lasso/logistic')])

# random forest
rfy = make_pipeline(transformer, RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
rfd = make_pipeline(transformer, RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
result = dml(X, D, y, rfy, rfd, nfolds=5, classifier=True)
table = pd.concat([table, summary(*result, X, D, y, name='random forest')])

# decision tree
dtry = make_pipeline(transformer, DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001))
dtrd = make_pipeline(transformer, DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001))
result = dml(X, D, y, dtry, dtrd, nfolds=5, classifier=True)
table = pd.concat([table, summary(*result, X, D, y, name='decision tree')])

# boosted trees
gbfy = make_pipeline(transformer, GradientBoostingRegressor(max_depth=2, n_iter_no_change=5))
gbfd = make_pipeline(transformer, GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
result = dml(X, D, y, gbfy, gbfd, nfolds=5, classifier=True)
table = pd.concat([table, summary(*result, X, D, y, name='boosted forest')])

# semi cross fitting: To avoid the computational cost of performing model selection within each fold (assuming that we don't select among an exponential set of hyperparameters/models in the number of samples), it is ok to perform model selection using all the data and then perform cross-fitting with the selected model
flamly = make_pipeline(transformer, AutoML(time_budget=100, task='regression', early_stop=True,
                                           eval_method='cv', n_splits=3, metric='r2', verbose=0))
flamld = make_pipeline(transformer, AutoML(time_budget=100, task='classification', early_stop=True,
                                           eval_method='cv', n_splits=3, metric='r2', verbose=0))
flamly.fit(X, y)
besty = make_pipeline(transformer, clone(flamly[-1].best_model_for_estimator(flamly[-1].best_estimator)))
flamld.fit(X, D)
bestd = make_pipeline(transformer, clone(flamld[-1].best_model_for_estimator(flamld[-1].best_estimator)))
result = dml(X, D, y, besty, bestd, nfolds=5, classifier=True)
table = pd.concat([table, summary(*result, X, D, y, name='automl (semi-cfit)')])

# semi cross fitting with stacking
def dml_dirty(X, D, y, modely_list, modeld_list, *,
              stacker=LinearRegression(), nfolds, classifier=False):
    '''
    DML for the Partially Linear Model setting with semi-cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely: the ML model for predicting the outcome y
    modeld: the ML model for predicting the treatment D
    stacker: model used to aggregate predictions of each of the base models
    nfolds: the number of folds in cross-fitting
    classifier: bool, whether the modeld is a classifier or a regressor

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the treatment D
    resy: the outcome residuals
    resD: the treatment residuals
    epsilon: the final residual-on-residual OLS regression residual
    '''
    # construct out-of-fold predictions for each model
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)
    yhats = np.array([cross_val_predict(modely, X, y, cv=cv, n_jobs=-1) for modely in modely_list]).T
    if classifier:
        Dhats = np.array([cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
                         for modeld in modeld_list]).T
    else:
        Dhats = np.array([cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1) for modeld in modeld_list]).T
    # calculate stacked residuals by finding optimal coefficients
    # and weigthing out-of-sample predictions by these coefficients
    yhat = stacker.fit(yhats, y).predict(yhats)
    Dhat = stacker.fit(Dhats, D).predict(Dhats)
    resy = y - yhat
    resD = D - Dhat
    # go with the stacked residuals
    point = np.mean(resy * resD) / np.mean(resD**2)
    epsilon = resy - point * resD
    var = np.mean(epsilon**2 * resD**2) / np.mean(resD**2)**2
    stderr = np.sqrt(var / X.shape[0])
    return point, stderr, yhat, Dhat, resy, resD, epsilon

result = dml_dirty(X, D, y, [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd],
                   nfolds=5, classifier=True)
table = pd.concat([table, summary(*result, X, D, y, name='stacked (semi-cfit)')])
table

env: PYTHONWARNINGS=ignore


,estimate,stderr,lower,upper,rmse y,rmse D,accuracy D
double lasso,9035.120004,1295.135748,6496.653938,11573.586070,54254.468883,0.443406,0.688553
lasso/logistic,9092.508157,1304.398170,6535.887743,11649.128571,54254.468883,0.444043,0.687847
random forest,8871.096519,1348.341577,6228.347029,11513.846010,54889.121007,0.444574,0.688553
decision tree,9236.195678,1440.551643,6412.714457,12059.676898,59427.392172,0.446437,0.688048
boosted forest,9110.713844,1343.747801,6476.968155,11744.459534,55657.022520,0.443533,0.690267
automl (semi-cfit),8877.435525,1300.756781,6327.952234,11426.918816,53908.841872,0.446314,0.689864
stacked (semi-cfit),8985.081489,1310.988123,6415.544768,11554.618210,54030.500545,0.442871,0.688956


In [10]:

data_bottom = data.loc[data['inc'] <= data['inc'].quantile(0.25)].copy()
data_top = data.loc[data['inc'] >= data['inc'].quantile(0.75)].copy()

# Define helper function to create X, D, y from a subset
def extract_XDy(df):
    """Given a subset of the 401k data, produce X, D, and y 
    consistent with the main analysis."""
    y_ = df['net_tfa'].values
    D_ = df['e401'].values
    X_ = df.drop([
        'e401', 'p401', 'a401', 'tw', 'tfa', 'net_tfa', 'tfa_he',
        'hval', 'hmort', 'hequity', 'nifa', 'net_nifa', 'net_n401',
        'ira', 'dum91', 'icat', 'ecat', 'zhat', 'i1', 'i2', 'i3',
        'i4', 'i5', 'i6', 'i7', 'a1', 'a2', 'a3', 'a4', 'a5'
    ], axis=1)
    return X_, D_, y_

X_bottom, D_bottom, y_bottom = extract_XDy(data_bottom)
X_top, D_top, y_top = extract_XDy(data_top)


# Create a helper function that runs all models and returns a summary table
def run_all_estimators(X, D, y, name_prefix=''):
    """Runs the pipeline of estimators and returns a summary results table."""
    table_local = []

    # cross-validation setup
    cv = KFold(n_splits=5, shuffle=True, random_state=123)
    
    # 1) double lasso
    lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
    lassod = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
    result = dml(X, D, y, lassoy, lassod, nfolds=5)
    table_local.append(summary(*result, X, D, y, name=f'{name_prefix} double lasso'))

    # 2) lasso/logistic
    lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
    lgrd = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
    result = dml(X, D, y, lassoy, lgrd, nfolds=5, classifier=True)
    table_local.append(summary(*result, X, D, y, name=f'{name_prefix} lasso/logistic'))

    # 3) random forest
    rfy = make_pipeline(transformer,
                        RandomForestRegressor(n_estimators=100,
                                              min_samples_leaf=10,
                                              ccp_alpha=0.001))
    rfd = make_pipeline(transformer,
                        RandomForestClassifier(n_estimators=100,
                                               min_samples_leaf=10,
                                               ccp_alpha=0.001))
    result = dml(X, D, y, rfy, rfd, nfolds=5, classifier=True)
    table_local.append(summary(*result, X, D, y, name=f'{name_prefix} random forest'))

    # 4) decision tree
    dtry = make_pipeline(transformer,
                         DecisionTreeRegressor(min_samples_leaf=10,
                                               ccp_alpha=0.001))
    dtrd = make_pipeline(transformer,
                         DecisionTreeClassifier(min_samples_leaf=10,
                                                ccp_alpha=0.001))
    result = dml(X, D, y, dtry, dtrd, nfolds=5, classifier=True)
    table_local.append(summary(*result, X, D, y, name=f'{name_prefix} decision tree'))

    # 5) boosted trees
    gbfy = make_pipeline(transformer,
                         GradientBoostingRegressor(max_depth=2,
                                                   n_iter_no_change=5))
    gbfd = make_pipeline(transformer,
                         GradientBoostingClassifier(max_depth=2,
                                                    n_iter_no_change=5))
    result = dml(X, D, y, gbfy, gbfd, nfolds=5, classifier=True)
    table_local.append(summary(*result, X, D, y, name=f'{name_prefix} boosted forest'))

    # 6) automl (semi cross-fitting)
    flamly = make_pipeline(transformer,
                           AutoML(time_budget=60,  # reduce if desired
                                  task='regression',
                                  early_stop=True,
                                  eval_method='cv',
                                  n_splits=3,
                                  metric='r2',
                                  verbose=0))
    flamld = make_pipeline(transformer,
                           AutoML(time_budget=60,
                                  task='classification',
                                  early_stop=True,
                                  eval_method='cv',
                                  n_splits=3,
                                  metric='r2',
                                  verbose=0))
    # Fit once on entire data
    flamly.fit(X, y)
    besty_model = flamly[-1].best_model_for_estimator(flamly[-1].best_estimator)
    besty = make_pipeline(transformer, clone(besty_model))

    flamld.fit(X, D)
    bestd_model = flamld[-1].best_model_for_estimator(flamld[-1].best_estimator)
    bestd = make_pipeline(transformer, clone(bestd_model))

    result = dml(X, D, y, besty, bestd, nfolds=5, classifier=True)
    table_local.append(summary(*result, X, D, y, name=f'{name_prefix} automl (semi-cfit)'))

    # 7) stacked (semi-cfit)
    # Re-use the same base models we created, but put them into lists:
    # - Notice we must re-create them fresh so they are unfitted before cross_val_predict
    #   or cross_val_predict won't do what we expect.
    lassoy_ = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
    lgrd_ = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
    rfy_ = make_pipeline(transformer,
                         RandomForestRegressor(n_estimators=100,
                                               min_samples_leaf=10,
                                               ccp_alpha=0.001))
    rfd_ = make_pipeline(transformer,
                         RandomForestClassifier(n_estimators=100,
                                                min_samples_leaf=10,
                                                ccp_alpha=0.001))
    dtry_ = make_pipeline(transformer,
                          DecisionTreeRegressor(min_samples_leaf=10,
                                                ccp_alpha=0.001))
    dtrd_ = make_pipeline(transformer,
                          DecisionTreeClassifier(min_samples_leaf=10,
                                                 ccp_alpha=0.001))
    gbfy_ = make_pipeline(transformer,
                          GradientBoostingRegressor(max_depth=2,
                                                    n_iter_no_change=5))
    gbfd_ = make_pipeline(transformer,
                          GradientBoostingClassifier(max_depth=2,
                                                     n_iter_no_change=5))

    modely_list = [lassoy_, rfy_, dtry_, gbfy_]
    modeld_list = [lgrd_, rfd_, dtrd_, gbfd_]

    result = dml_dirty(X, D, y, modely_list, modeld_list,
                       stacker=LinearRegression(),
                       nfolds=5, classifier=True)
    table_local.append(summary(*result, X, D, y, name=f'{name_prefix} stacked (semi-cfit)'))

    # Concatenate all results
    return pd.concat(table_local)


table_bottom = run_all_estimators(X_bottom, D_bottom, y_bottom, name_prefix='bottom25%')
table_top = run_all_estimators(X_top, D_top, y_top, name_prefix='top25%')

print("=== Bottom 25% Income Sample ===")
print(table_bottom)

print("\n=== Top 25% Income Sample ===")
print(table_top)

=== Bottom 25% Income Sample ===
                                  estimate       stderr        lower  \
bottom25% double lasso         3769.763582  1092.945664  1627.590080   
bottom25% lasso/logistic       3803.759038  1072.578658  1701.504869   
bottom25% random forest        4391.212252  1092.834491  2249.256649   
bottom25% decision tree        3470.960704  1032.092136  1448.060118   
bottom25% boosted forest       3937.557332  1095.544442  1790.290227   
bottom25% automl (semi-cfit)   3916.284487  1090.867865  1778.183471   
bottom25% stacked (semi-cfit)  4020.224735  1085.316134  1893.005112   

                                     upper        rmse y    rmse D  accuracy D  
bottom25% double lasso         5911.937083  13400.361810  0.343801    0.846433  
bottom25% lasso/logistic       5906.013207  13400.361810  0.354799    0.844015  
bottom25% random forest        6533.167855  13504.626992  0.345757    0.846030  
bottom25% decision tree        5493.861290  14728.635257  0.380351

For the PLR setting, there indeed seems to be heterogeneity in the treatment with respect to income, with the bottom 25% of earners seeing estimates around 3.8k and the top 25% seeing estimates of around 17.5k. The different machine learning models are broadly consistent across all three income groups.  
  
Next we more to the IRM part:

In [11]:
def dr(X, D, y, modely0, modely1, modeld, *, trimming=0.01, nfolds):
    '''
    DML for the Interactive Regression Model setting (Doubly Robust Learning)
    with cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely0: the ML model for predicting the outcome y in the control population
    modely1: the ML model for predicting the outcome y in the treated population
    modeld: the ML model for predicting the treatment D
    trimming: threshold below which to trim propensities
    nfolds: the number of folds in cross-fitting

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the outcome D
    resy: the outcome residuals
    resD: the treatment residuals
    drhat: the doubly robust quantity for each sample
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)
    yhat0, yhat1 = np.zeros(y.shape), np.zeros(y.shape)
    # we will fit a model E[Y| D, X] by fitting a separate model for D==0
    # and a separate model for D==1.
    for train, test in cv.split(X, y):
        # train a model on training data that received treatment zero and predict on all data in test set
        yhat0[test] = clone(modely0).fit(X.iloc[train][D[train] == 0], y[train][D[train] == 0]).predict(X.iloc[test])
        # train a model on training data that received treatment one and predict on all data in test set
        yhat1[test] = clone(modely1).fit(X.iloc[train][D[train] == 1], y[train][D[train] == 1]).predict(X.iloc[test])
    # prediction for observed treatment
    yhat = yhat0 * (1 - D) + yhat1 * D
    # propensity scores
    Dhat = cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
    Dhat = np.clip(Dhat, trimming, 1 - trimming)
    # doubly robust quantity for every sample
    drhat = yhat1 - yhat0 + (y - yhat) * (D / Dhat - (1 - D) / (1 - Dhat))
    point = np.mean(drhat)
    var = np.var(drhat)
    stderr = np.sqrt(var / X.shape[0])
    return point, stderr, yhat, Dhat, y - yhat, D - Dhat, drhat

v = KFold(n_splits=5, shuffle=True, random_state=123)
lassoytest = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
lgrdtest = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
result = dr(X, D, y, lassoytest, lassoytest, lgrdtest, nfolds=5)
seed_estimates = summary(*result, X, D, y, name='lasso/logistic')

for i in range(9):
    cv = KFold(n_splits=5, shuffle=True, random_state=i)
    lassoytest = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
    lgrdtest = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
    result = dr(X, D, y, lassoytest, lassoytest, lgrdtest, nfolds=5)
    seed_estimates = pd.concat([seed_estimates, summary(*result, X, D, y, name='lasso/logistic')])

med_theta = np.median(seed_estimates.values[:, 0])
se_med = np.sqrt(np.median((seed_estimates.values[:, 1])**2 + (seed_estimates.values[:, 0] - med_theta)**2))
tabledr = pd.DataFrame({'estimate': med_theta,
                        'stderr': se_med,
                        'lower': med_theta - 1.96 * se_med,
                        'upper': med_theta + 1.96 * se_med,
                        'rmse y': np.median(seed_estimates.values[:, 4]),
                        'rmse D': np.median(seed_estimates.values[:, 5]),
                        'accuracy D': np.median(seed_estimates.values[:, 6]),
                        }, index=['lasso/logistic'])

rfy = make_pipeline(transformer, RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
rfd = make_pipeline(transformer, RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
result = dr(X, D, y, rfy, rfy, rfd, nfolds=5)
tabledr = pd.concat([tabledr, summary(*result, X, D, y, name='random forest')])
dtry = make_pipeline(transformer, DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001))
dtrd = make_pipeline(transformer, DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001))
result = dr(X, D, y, dtry, dtry, dtrd, nfolds=5)
tabledr = pd.concat([tabledr, summary(*result, X, D, y, name='decision tree')])
gbfy = make_pipeline(transformer, GradientBoostingRegressor(max_depth=2, n_iter_no_change=5))
gbfd = make_pipeline(transformer, GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
result = dr(X, D, y, gbfy, gbfy, gbfd, nfolds=5)
tabledr = pd.concat([tabledr, summary(*result, X, D, y, name='boosted forest')])
# semi cross-fitting
flamly0 = make_pipeline(transformer, AutoML(time_budget=60, task='regression', early_stop=True,
                                            eval_method='cv', n_splits=3, metric='r2', verbose=0))
flamly1 = make_pipeline(transformer, AutoML(time_budget=60, task='regression', early_stop=True,
                                            eval_method='cv', n_splits=3, metric='r2', verbose=0))
flamld = make_pipeline(transformer, AutoML(time_budget=60, task='classification', early_stop=True,
                                           eval_method='cv', n_splits=3, metric='r2', verbose=0))

flamly0.fit(X[D == 0], y[D == 0])
besty0 = make_pipeline(transformer, clone(flamly0[-1].best_model_for_estimator(flamly0[-1].best_estimator)))
flamly1.fit(X[D == 1], y[D == 1])
besty1 = make_pipeline(transformer, clone(flamly1[-1].best_model_for_estimator(flamly1[-1].best_estimator)))
flamld.fit(X, D)
bestd = make_pipeline(transformer, clone(flamld[-1].best_model_for_estimator(flamld[-1].best_estimator)))
result = dr(X, D, y, besty0, besty1, bestd, nfolds=5)
tabledr = pd.concat([tabledr, summary(*result, X, D, y, name='automl (semi-cfit)')])
def dr_dirty(X, D, y, modely0_list, modely1_list, modeld_list, *,
             stacker=LinearRegression(), trimming=0.01, nfolds):
    '''
    DML for the Interactive Regression Model setting (Doubly Robust Learning)
    with cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely_list: list of ML models for predicting the outcome y
    modeld_list: list of ML models for predicting the treatment D
    stacker: model used to aggregate predictions of each of the base models
    trimming: threshold below which to trim propensities
    nfolds: the number of folds in cross-fitting

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the outcome D
    resy: the outcome residuals
    resD: the treatment residuals
    drhat: the doubly robust quantity for each sample
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)

    # we will fit a model E[Y| D, X] by fitting a separate model for D==0
    # and a separate model for D==1. We do that for each model type in modely_list
    yhats0, yhats1 = np.zeros((y.shape[0], len(modely0_list))), np.zeros((y.shape[0], len(modely1_list)))
    for train, test in cv.split(X, y):
        for it, modely0 in enumerate(modely0_list):
            mdl = clone(modely0).fit(X.iloc[train][D[train] == 0], y[train][D[train] == 0])
            yhats0[test, it] = mdl.predict(X.iloc[test])
        for it, modely1 in enumerate(modely1_list):
            mdl = clone(modely1).fit(X.iloc[train][D[train] == 1], y[train][D[train] == 1])
            yhats1[test, it] = mdl.predict(X.iloc[test])

    # calculate stacking weights for the outcome model for each population
    # and combine the outcome model predictions
    yhat0 = clone(stacker).fit(yhats0[D == 0], y[D == 0]).predict(yhats0)
    yhat1 = clone(stacker).fit(yhats1[D == 1], y[D == 1]).predict(yhats1)

    # prediction for observed treatment using the stacked model
    yhat = yhat0 * (1 - D) + yhat1 * D

    # propensity scores
    Dhats = np.array([cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
                     for modeld in modeld_list]).T
    # construct coefficients on each model based on stacker
    Dhat = clone(stacker).fit(Dhats, D).predict(Dhats)
    # trim propensities
    Dhat = np.clip(Dhat, trimming, 1 - trimming)

    # doubly robust quantity for every sample
    drhat = yhat1 - yhat0 + (y - yhat) * (D / Dhat - (1 - D) / (1 - Dhat))
    point = np.mean(drhat)
    var = np.var(drhat)
    stderr = np.sqrt(var / X.shape[0])
    return point, stderr, yhat, Dhat, y - yhat, D - Dhat, drhat

result = dr_dirty(X, D, y, [lassoy, rfy, dtry, gbfy], [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd], nfolds=5)
tabledr = pd.concat([tabledr, summary(*result, X, D, y, name='stacked (semi-cfit)')])
tabledr


,estimate,stderr,lower,upper,rmse y,rmse D,accuracy D
lasso/logistic,7726.585781,1159.322663,5454.313361,9998.858202,54060.702446,0.444041,0.687948
random forest,7777.909708,1146.768056,5530.244318,10025.575099,55592.303235,0.444747,0.689057
decision tree,7846.333657,1255.457709,5385.636547,10307.030767,60491.245996,0.446437,0.688048
boosted forest,8605.121529,1148.964341,6353.151421,10857.091638,55118.774981,0.443754,0.688855
automl (semi-cfit),8493.401447,1098.094834,6341.135573,10645.667321,53750.922780,0.448531,0.688250
stacked (semi-cfit),7762.670926,1127.006260,5553.738656,9971.603197,53406.542825,0.442790,0.689662


In [12]:
data_bottom = data.query('inc < inc.quantile(.25)').copy()
data_top    = data.query('inc > inc.quantile(.75)').copy()

def extract_XDy(df):
    """Given a subset of the 401k data, produce X, D, and y 
    consistent with the main DR analysis."""
    y_ = df['net_tfa'].values
    D_ = df['e401'].values
    X_ = df.drop([
        'e401', 'p401', 'a401', 'tw', 'tfa', 'net_tfa', 'tfa_he',
        'hval', 'hmort', 'hequity', 'nifa', 'net_nifa', 'net_n401',
        'ira', 'dum91', 'icat', 'ecat', 'zhat', 'i1', 'i2', 'i3',
        'i4', 'i5', 'i6', 'i7', 'a1', 'a2', 'a3', 'a4', 'a5'
    ], axis=1)
    return X_, D_, y_

X_bottom, D_bottom, y_bottom = extract_XDy(data_bottom)
X_top, D_top, y_top          = extract_XDy(data_top)

def dr(X, D, y, modely0, modely1, modeld, *, trimming=0.01, nfolds=5):
    '''
    DML for the Interactive Regression Model setting (Doubly Robust Learning)
    with cross-fitting
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)
    yhat0, yhat1 = np.zeros(y.shape), np.zeros(y.shape)
    for train, test in cv.split(X, y):
        # Fit E[Y|D=0,X]
        mdl0 = clone(modely0).fit(X.iloc[train][D[train] == 0], y[train][D[train] == 0])
        yhat0[test] = mdl0.predict(X.iloc[test])
        # Fit E[Y|D=1,X]
        mdl1 = clone(modely1).fit(X.iloc[train][D[train] == 1], y[train][D[train] == 1])
        yhat1[test] = mdl1.predict(X.iloc[test])

    # Combine to get E[Y|D,X] predictions for the observed D
    yhat = yhat0 * (1 - D) + yhat1 * D

    # Propensity scores
    Dhat = cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
    Dhat = np.clip(Dhat, trimming, 1 - trimming)

    # Doubly robust quantity for each sample
    drhat = yhat1 - yhat0 + (y - yhat) * (D / Dhat - (1 - D) / (1 - Dhat))
    point = np.mean(drhat)
    var   = np.var(drhat)
    stderr= np.sqrt(var / X.shape[0])

    # Return:
    #   - point: the point estimate
    #   - stderr: standard error
    #   - yhat: cross-fitted predictions for the observed D
    #   - Dhat: cross-fitted propensities
    #   - (y - yhat): outcome residual
    #   - (D - Dhat): treatment residual
    #   - drhat: doubly robust terms
    return point, stderr, yhat, Dhat, (y - yhat), (D - Dhat), drhat

def dr_dirty(
    X, D, y,
    modely0_list, modely1_list, modeld_list,
    stacker=LinearRegression(), trimming=0.01, nfolds=5
):
    '''
    DML for the Interactive Regression Model setting (Doubly Robust Learning)
    with cross-fitting and stacking
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)

    # Prepare space for multiple model predictions
    yhats0 = np.zeros((X.shape[0], len(modely0_list)))
    yhats1 = np.zeros((X.shape[0], len(modely1_list)))

    # Cross-fitting: fit each base model E[Y|D=0,X], E[Y|D=1,X]
    for train, test in cv.split(X, y):
        for i_m0, m0 in enumerate(modely0_list):
            m0_cl = clone(m0).fit(X.iloc[train][D[train] == 0], y[train][D[train] == 0])
            yhats0[test, i_m0] = m0_cl.predict(X.iloc[test])
        for i_m1, m1 in enumerate(modely1_list):
            m1_cl = clone(m1).fit(X.iloc[train][D[train] == 1], y[train][D[train] == 1])
            yhats1[test, i_m1] = m1_cl.predict(X.iloc[test])

    # Stack them for D=0 predictions
    yhat0 = clone(stacker).fit(yhats0[D == 0], y[D == 0]).predict(yhats0)
    # Stack them for D=1 predictions
    yhat1 = clone(stacker).fit(yhats1[D == 1], y[D == 1]).predict(yhats1)
    # Combined model prediction for Y
    yhat = yhat0 * (1 - D) + yhat1 * D

    # Now do the same for propensity models
    Dhats_list = []
    for md in modeld_list:
        Dhats_list.append(cross_val_predict(md, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1])
    Dhats_arr = np.vstack(Dhats_list).T  # shape: (n_samples, n_models)

    # Stack the predicted propensity
    Dhat = clone(stacker).fit(Dhats_arr, D).predict(Dhats_arr)
    Dhat = np.clip(Dhat, trimming, 1 - trimming)

    # Doubly robust quantity
    drhat = yhat1 - yhat0 + (y - yhat) * (D / Dhat - (1 - D) / (1 - Dhat))
    point = np.mean(drhat)
    var   = np.var(drhat)
    stderr= np.sqrt(var / X.shape[0])

    return point, stderr, yhat, Dhat, (y - yhat), (D - Dhat), drhat


def summary(point, stderr, yhat, Dhat, resy, resD, drhat, X, D, y, *, name=''):
    return pd.DataFrame({
        'estimate': point,  # point estimate
        'stderr': stderr,   # standard error
        'lower': point - 1.96 * stderr,
        'upper': point + 1.96 * stderr,
        'rmse y': np.sqrt(np.mean(resy**2)),
        'rmse D': np.sqrt(np.mean(resD**2)),
        # classification accuracy for D, if it's binary:
        'accuracy D': np.mean(np.abs(resD) < 0.5),
    }, index=[name])


def run_dr_analysis(X, D, y):
    """
    Replicates the DR analysis shown in the original code:
      - lasso/logistic with repeated seeds, then median-based estimate
      - random forest
      - decision tree
      - boosted forest
      - automl (semi-cfit)
      - stacking
    Returns a single table of results.
    """
    # -- 1) Lasso/logistic repeated over seeds, then median-based estimate
    seed_estimates = None

    # We'll define a base 5-fold for the model pipelines:
    cv_5fold = KFold(n_splits=5, shuffle=True, random_state=123)

    # One example run with seed=123
    lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv_5fold))
    lgrd   = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv_5fold))

    # DR for single seed first
    result = dr(X, D, y, lassoy, lassoy, lgrd, nfolds=5)
    seed_estimates = summary(*result, X, D, y, name='lasso/logistic')

    # Loop over multiple seeds
    for i in range(9):
        cv_seed = KFold(n_splits=5, shuffle=True, random_state=i)
        lassoy_i = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv_seed))
        lgrd_i   = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv_seed))

        result_i = dr(X, D, y, lassoy_i, lassoy_i, lgrd_i, nfolds=5)
        seed_estimates = pd.concat([seed_estimates, summary(*result_i, X, D, y, name='lasso/logistic')])

    # Compute median-based point estimate and standard error
    med_theta  = np.median(seed_estimates['estimate'].values)
    # sqrt(median(var_i) + var( point_i ))
    # but the original code basically does:
    se_med     = np.sqrt(
        np.median(seed_estimates['stderr'].values ** 2)
        + np.median((seed_estimates['estimate'].values - med_theta) ** 2)
    )
    tabledr = pd.DataFrame({
        'estimate': med_theta,
        'stderr':   se_med,
        'lower':    med_theta - 1.96 * se_med,
        'upper':    med_theta + 1.96 * se_med,
        'rmse y':   np.median(seed_estimates['rmse y'].values),
        'rmse D':   np.median(seed_estimates['rmse D'].values),
        'accuracy D': np.median(seed_estimates['accuracy D'].values)
    }, index=['lasso/logistic'])

    # -- 2) Random Forest
    rfy = make_pipeline(transformer, 
                        RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
    rfd = make_pipeline(transformer, 
                        RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
    result = dr(X, D, y, rfy, rfy, rfd, nfolds=5)
    tabledr = pd.concat([tabledr, summary(*result, X, D, y, name='random forest')])

    # -- 3) Decision Tree
    dtry = make_pipeline(transformer,
                         DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001))
    dtrd = make_pipeline(transformer,
                         DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001))
    result = dr(X, D, y, dtry, dtry, dtrd, nfolds=5)
    tabledr = pd.concat([tabledr, summary(*result, X, D, y, name='decision tree')])

    # -- 4) Boosted Forest
    gbfy = make_pipeline(transformer, 
                         GradientBoostingRegressor(max_depth=2, n_iter_no_change=5))
    gbfd = make_pipeline(transformer, 
                         GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
    result = dr(X, D, y, gbfy, gbfy, gbfd, nfolds=5)
    tabledr = pd.concat([tabledr, summary(*result, X, D, y, name='boosted forest')])

    # -- 5) AutoML (semi cross-fitting)
    flamly0 = make_pipeline(transformer,
                            AutoML(time_budget=60, task='regression', early_stop=True,
                                   eval_method='cv', n_splits=3, metric='r2', verbose=0))
    flamly1 = make_pipeline(transformer,
                            AutoML(time_budget=60, task='regression', early_stop=True,
                                   eval_method='cv', n_splits=3, metric='r2', verbose=0))
    flamld  = make_pipeline(transformer,
                            AutoML(time_budget=60, task='classification', early_stop=True,
                                   eval_method='cv', n_splits=3, metric='r2', verbose=0))

    # Fit Y|D=0, Y|D=1 on subsets
    flamly0.fit(X[D == 0], y[D == 0])
    besty0_model = flamly0[-1].best_model_for_estimator(flamly0[-1].best_estimator)
    besty0 = make_pipeline(transformer, clone(besty0_model))

    flamly1.fit(X[D == 1], y[D == 1])
    besty1_model = flamly1[-1].best_model_for_estimator(flamly1[-1].best_estimator)
    besty1 = make_pipeline(transformer, clone(besty1_model))

    # Fit propensities on the full sample
    flamld.fit(X, D)
    bestd_model = flamld[-1].best_model_for_estimator(flamld[-1].best_estimator)
    bestd = make_pipeline(transformer, clone(bestd_model))

    result = dr(X, D, y, besty0, besty1, bestd, nfolds=5)
    tabledr = pd.concat([tabledr, summary(*result, X, D, y, name='automl (semi-cfit)')])

    # -- 6) Stacked (semi-cfit)
    # Prepare lists of base models for Y|D=0, Y|D=1
    lassoy_ = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv_5fold))
    rfy_    = make_pipeline(transformer, 
                            RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
    dtry_   = make_pipeline(transformer, 
                            DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001))
    gbfy_   = make_pipeline(transformer, 
                            GradientBoostingRegressor(max_depth=2, n_iter_no_change=5))

    modely0_list = [lassoy_, rfy_, dtry_, gbfy_]  # for D=0
    modely1_list = [lassoy_, rfy_, dtry_, gbfy_]  # for D=1

    lgrd_   = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv_5fold))
    rfd_    = make_pipeline(transformer, 
                            RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
    dtrd_   = make_pipeline(transformer, 
                            DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001))
    gbfd_   = make_pipeline(transformer, 
                            GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
    modeld_list = [lgrd_, rfd_, dtrd_, gbfd_]

    result = dr_dirty(X, D, y, modely0_list, modely1_list, modeld_list, stacker=LinearRegression(), nfolds=5)
    tabledr = pd.concat([tabledr, summary(*result, X, D, y, name='stacked (semi-cfit)')])

    return tabledr

table_dr_bottom = run_dr_analysis(X_bottom, D_bottom, y_bottom)
table_dr_top    = run_dr_analysis(X_top,    D_top,    y_top   )

print("=== Doubly Robust Results: Bottom 25% Income ===")
print(table_dr_bottom)

print("\n=== Doubly Robust Results: Top 25% Income ===")
print(table_dr_top)

=== Doubly Robust Results: Bottom 25% Income ===
                        estimate       stderr        lower         upper  \
lasso/logistic       4168.663113  1055.478209  2099.925824   6237.400403   
random forest        3799.230032   991.820440  1855.261969   5743.198095   
decision tree        6575.695867  3065.986166   566.362982  12585.028752   
boosted forest       4743.873292  1050.278674  2685.327092   6802.419492   
automl (semi-cfit)   4513.585430  1017.811332  2518.675220   6508.495641   
stacked (semi-cfit)  4132.274210   993.706517  2184.609436   6079.938984   

                           rmse y    rmse D  accuracy D  
lasso/logistic       13391.064772  0.357524    0.841001  
random forest        13456.578497  0.346027    0.845843  
decision tree        14583.602725  0.370841    0.809927  
boosted forest       13537.975130  0.344690    0.844633  
automl (semi-cfit)   13170.419927  0.346929    0.839790  
stacked (semi-cfit)  13227.455519  0.343537    0.844229  

=== Doubly 

In the IRM setting, we again see heterogeneity in the treatment effect with respect to income, with the bottom 25% of earners seeing estimates around 4.5k and the top 25% seeing estimates of around 17k. The different machine learning models are broadly consistent across all three income groups with the exception of the decision tree, which has far higher estimates and standard errors than other methods.  
  
The IRM and PLR results differ slightly in terms of the estimate, but both find heterogeneity and are broadly in the same range of values.

Now we move to implemetting semi-crossfitting with best model selection, first for PLR and then for IRM.

In [13]:
# b.)
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import mean_squared_error
from copy import deepcopy

def get_oof_and_mse(model, X, y, cv, classifier=False):
    """
    Returns out-of-fold predictions and the MSE (or 'regression MSE' if classifier=True).
    If classifier=True, we use model.predict_proba(...).
    """
    if classifier:
        # For binary D, treat the problem as a regression on [0,1], 
        # so we measure MSE on predicted probabilities
        preds = cross_val_predict(model, X, y, cv=cv, method='predict_proba')[:, 1]
    else:
        preds = cross_val_predict(model, X, y, cv=cv)
    mse_val = mean_squared_error(y, preds)
    return preds, mse_val


def dml_select_best(X, D, y, modely_list, modeld_list, *, nfolds=5, classifier=False, trimming=0.01):
    """
    Semi-cross-fitting for the Partially Linear Model (PLR) by selecting
    the single best model for Y and single best model for D among user-provided lists.

    Steps:
      1) For each candidate in modely_list, get OOF predictions vs. y. Pick the best by MSE.
      2) For each candidate in modeld_list, get OOF predictions vs. D. Pick the best by MSE.
         If classifier=True, each model in modeld_list is treated as a classifier,
         but we still measure MSE on the predicted probability vs. the true D.
      3) Use the chosen best model's OOF predictions for yhat and Dhat.
      4) Compute partial linear estimate as in the usual DML formula:
            theta = E[(y - yhat)*(D - Dhat)] / E[(D - Dhat)^2]
         and the standard error formula.

    Returns:
      point, stderr, yhat, Dhat, resy, resD, epsilon
    """
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)

    # --- 1) Select best model for y
    best_mse_y = np.inf
    best_preds_y = None
    best_model_y = None

    for candidate in modely_list:
        # Clone so we don't pollute the original pipeline with partial fits
        cand = deepcopy(candidate)
        preds, mse_val = get_oof_and_mse(cand, X, y, cv, classifier=False)
        if mse_val < best_mse_y:
            best_mse_y = mse_val
            best_preds_y = preds
            best_model_y = cand

    # --- 2) Select best model for D
    best_mse_d = np.inf
    best_preds_d = None
    best_model_d = None

    for candidate in modeld_list:
        # If classifier=True, we measure MSE on predicted probabilities
        cand = deepcopy(candidate)
        preds, mse_val = get_oof_and_mse(cand, X, D, cv, classifier=classifier)
        if mse_val < best_mse_d:
            best_mse_d = mse_val
            best_preds_d = preds
            best_model_d = cand

    # Residuals
    resy = y - best_preds_y
    resD = D - best_preds_d

    # Final partial linear estimate
    point = np.mean(resy * resD) / np.mean(resD**2)
    epsilon = resy - point * resD

    var = np.mean(epsilon**2 * resD**2) / (np.mean(resD**2)**2)
    stderr = np.sqrt(var / X.shape[0])

    return point, stderr, best_preds_y, best_preds_d, resy, resD, epsilon

modely_list = [lassoy, rfy, dtry, gbfy]
modeld_list = [lgrd, rfd, dtrd, gbfd]

point, stderr, yhat, Dhat, resy, resD, epsilon = dml_select_best(
    X, D, y, modely_list, modeld_list, nfolds=5, classifier=True
)

table_select = summary(
    point, stderr, yhat, Dhat, resy, resD, epsilon, X, D, y,
    name='select-best (semi-cfit) PLR'
)
point_bottom, stderr_bottom, yhat_bottom, Dhat_bottom, resy_bottom, resD_bottom, epsilon_bottom = dml_select_best(
    X_bottom, D_bottom, y_bottom, modely_list, modeld_list, nfolds=5, classifier=True
)
table_select_bottom = summary(
    point_bottom, stderr_bottom, yhat_bottom, Dhat_bottom, resy_bottom, resD_bottom, epsilon_bottom, X_bottom, D_bottom, y_bottom,
    name='select-best (semi-cfit) PLR (bottom 25%)'
)
table_select = pd.concat([table_select, table_select_bottom])
point_top, stderr_top, yhat_top, Dhat_top, resy_top, resD_top, epsilon_top = dml_select_best(
    X_top, D_top, y_top, modely_list, modeld_list, nfolds=5, classifier=True
)
table_select_top = summary(
    point_top, stderr_top, yhat_top, Dhat_top, resy_top, resD_top, epsilon_top, X_top, D_top, y_top,
    name='select-best (semi-cfit) PLR (top 25%)'
)
table_select = pd.concat([table_select, table_select_top])
table_select

,estimate,stderr,lower,upper,rmse y,rmse D,accuracy D
select-best (semi-cfit) PLR,8834.698468,1321.460893,6244.635118,11424.761817,54254.468883,0.443139,0.691578
select-best (semi-cfit) PLR (bottom 25%),3999.925670,1124.172083,1796.548387,6203.302954,13398.184779,0.343868,0.844633
select-best (semi-cfit) PLR (top 25%),18204.194590,3867.405524,10624.079763,25784.309418,91393.039963,0.482708,0.601049


For IRM:

In [14]:
def get_oof_and_mse_irm(model, X, y, D, which_d, cv):
    """
    For IRM, we measure OOF performance only on the subset where D == which_d.
    We do cross-fitting: train on all 'train' points that have D==which_d, 
    predict on the entire test fold, but compute MSE only for the test fold 
    members that also have D==which_d.
    
    Returns OOF predictions (full length, but only truly valid for D==which_d),
    plus MSE measured on that subset.
    """
    preds_full = np.zeros(len(y), dtype=float)
    for train_idx, test_idx in cv.split(X, y):
        # filter the training portion to only those with D==which_d
        train_sub = train_idx[D[train_idx] == which_d]
        # Fit on (X, y) for that sub-sample
        model_cl = deepcopy(model).fit(X.iloc[train_sub], y[train_sub])
        # Predict on the entire test fold
        preds_full[test_idx] = model_cl.predict(X.iloc[test_idx])
    # MSE only on the subset that has D==which_d
    mse_val = mean_squared_error(y[D==which_d], preds_full[D==which_d])
    return preds_full, mse_val

def dr_select_best(X, D, y, modely0_list, modely1_list, modeld_list,
                   trimming=0.01, nfolds=5):
    """
    Doubly-Robust (IRM) with semi-cross-fitting:
    Select single best model for Y|D=0 from modely0_list,
    single best model for Y|D=1 from modely1_list,
    single best model for the propensity from modeld_list (all data).
    
    Then run the standard cross-fitting formula to construct:
      yhat0, yhat1, Dhat, drhat,
    and output the usual IRM results.
    """
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)

    # --- 1) find best model for Y|D=0
    best_mse_y0 = np.inf
    best_preds_y0 = None
    best_model_y0 = None
    for candidate in modely0_list:
        preds0, mse0 = get_oof_and_mse_irm(candidate, X, y, D, which_d=0, cv=cv)
        if mse0 < best_mse_y0:
            best_mse_y0 = mse0
            best_preds_y0 = preds0
            best_model_y0 = candidate

    # --- 2) find best model for Y|D=1
    best_mse_y1 = np.inf
    best_preds_y1 = None
    best_model_y1 = None
    for candidate in modely1_list:
        preds1, mse1 = get_oof_and_mse_irm(candidate, X, y, D, which_d=1, cv=cv)
        if mse1 < best_mse_y1:
            best_mse_y1 = mse1
            best_preds_y1 = preds1
            best_model_y1 = candidate

    # --- 3) find best model for D (propensity), measure MSE on entire sample
    #         but we treat D as binary, using predicted probability
    cv2 = KFold(n_splits=nfolds, shuffle=True, random_state=123)
    best_mse_d = np.inf
    best_preds_d = None
    best_model_d = None
    for candidate in modeld_list:
        preds_d, mse_d = get_oof_and_mse(candidate, X, D, cv2, classifier=True)
        if mse_d < best_mse_d:
            best_mse_d = mse_d
            best_preds_d = preds_d
            best_model_d = candidate

    # --- 4) IRM formula
    # We already have cross-fitted yhat0, yhat1, Dhat = best_preds_d
    yhat = best_preds_y0*(1 - D) + best_preds_y1*D
    Dhat = np.clip(best_preds_d, trimming, 1 - trimming)

    # DR score
    drhat = best_preds_y1 - best_preds_y0 + (y - yhat) * (
        D / Dhat - (1 - D)/(1 - Dhat)
    )
    point = np.mean(drhat)
    var = np.var(drhat)
    stderr = np.sqrt(var / X.shape[0])

    return (
        point,
        stderr,
        yhat,                # cross-fitted E[Y|D=observed, X]
        Dhat,                # cross-fitted e(X)
        (y - yhat),          # residual in Y space
        (D - Dhat),          # residual in D space
        drhat
    )
    
modely0_list = [lassoy, rfy, dtry, gbfy]
modely1_list = [lassoy, rfy, dtry, gbfy]
modeld_list  = [lgrd,  rfd,  dtrd, gbfd]

res_all = dr_select_best(
    X, D, y, modely0_list, modely1_list, modeld_list, nfolds=5
)
res_bottom_25 = dr_select_best(
    X_bottom, D_bottom, y_bottom, modely0_list, modely1_list, modeld_list, nfolds=5
)
res_top_25 = dr_select_best(
    X_top, D_top, y_top, modely1_list, modely0_list, modeld_list, nfolds=5
)
table_all = summary(*res_all, X, D, y, name="select best IRM with semi cross fitting all samples")
table_bottom_25 = summary(*res_bottom_25, X_bottom, D_bottom, y_bottom, name=f"select best IRM with semi cross fitting bottom 25\% income")
table_top_25 = summary(*res_top_25, X_top, D_top, y_top, name=f"select best IRM with semi cross fitting top 25\% income")

table_final = pd.concat([table_all, table_bottom_25, table_top_25])
table_final

,estimate,stderr,lower,upper,rmse y,rmse D,accuracy D
select best IRM with semi cross fitting all samples,7718.844897,1119.692846,5524.246919,9913.442874,54026.928780,0.443322,0.689864
select best IRM with semi cross fitting bottom 25\% income,4243.618356,1107.527383,2072.864686,6414.372026,13378.218781,0.344798,0.845036
select best IRM with semi cross fitting top 25\% income,18393.721922,3820.594388,10905.356922,25882.086922,91533.229313,0.482708,0.601049


In [15]:
 #c.) 
W = StandardScaler().fit_transform(transformer.fit_transform(X))
# PLR in econml
# ! pip install econml
from econml.dml import LinearDML


# double lasso in econml
cv = KFold(n_splits=5, shuffle=True, random_state=123)
ldml_lasso = LinearDML(
    model_y=LassoCV(cv=cv),
    model_t=LassoCV(cv=cv),
).fit(y, D, W=W)
print("----------------- Double Lasso in EconML for PLR------------")
print(ldml_lasso.summary())

# random forest in econml
ldml_rf = LinearDML(
    model_y=RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    model_t=RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    cv=5,
    discrete_treatment=True,
    random_state=123
).fit(y, D, W=W)
print("-----------------Random Forest in EconML for PLR------------")
print(ldml_rf.summary())

# gradient boosting in econml
ldml_gb = LinearDML(
    model_y=GradientBoostingRegressor(max_depth=2, n_iter_no_change=5, random_state=123),
    model_t=GradientBoostingClassifier(max_depth=2, n_iter_no_change=5, random_state=123),
    cv=5,
    discrete_treatment=True,
    random_state=123
).fit(y, D, W=W)
print("-----------------Gradient Boosting in EconML for PLR------------")
print(ldml_gb.summary())

# PLR with Decision Tree
ldml_dt = LinearDML(
    model_y=DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    model_t=DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    cv=5,
    discrete_treatment=True,
    random_state=123
).fit(y, D, W=W)
print("-----------------Decision Tree in EconML for PLR------------")
print(ldml_dt.summary())

# plr in double ml
# ! pip install doubleml
from doubleml import DoubleMLData
import doubleml as dbml


dml_data = DoubleMLData.from_arrays(W, y, D)

try:
    # double lasso
    cv = KFold(n_splits=5, shuffle=True, random_state=123)
    dml_plr_lasso = dbml.DoubleMLPLR(
        dml_data,
        LassoCV(cv=cv),
        LassoCV(cv=cv),
        n_folds=5,
    )
    dml_plr_lasso.fit()
    print("-----------------Double Lasso in DoubleML for PLR------------")
    # random forest
    dml_plr_rf = dbml.DoubleMLPLR(
        dml_data,
        RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001, random_state=123),
        RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001, random_state=123),
        n_folds=5,
    )
    dml_plr_rf.fit()
    print("-----------------Random Forest in DoubleML for PLR------------")
    print(dml_plr_rf.summary)
except Exception as e:
    print(e)
    print("DoubleML failed to run the PLR models")

# decision tree
dml_plr_dt = dbml.DoubleMLPLR(
    dml_data,
    DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    n_folds=5,
)

dml_plr_dt.fit()
print("-----------------Decision Tree in DoubleML for PLR------------")
print(dml_plr_dt.summary)

# gradient boosting

dml_plr_gb = dbml.DoubleMLPLR(
    dml_data,
    GradientBoostingRegressor(max_depth=2, n_iter_no_change=5, random_state=123),
    GradientBoostingClassifier(max_depth=2, n_iter_no_change=5, random_state=123),
    n_folds=5,
)

dml_plr_gb.fit() 
print("-----------------Gradient Boosting in DoubleML for PLR------------")
print(dml_plr_gb.summary)


# irm with econml

from econml.dr import LinearDRLearner   
from econml.utilities import SeparateModel

# random forest
dr_forest = LinearDRLearner(
    model_regression=RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    model_propensity=RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    cv=5,
)
dr_forest.fit(y, D, W=W)
print("-----------------Random Forest in EconML for IRM------------")
print(dr_forest.summary(T=1))

# random forest using seperate models for model_regression
dr_forest_sep = LinearDRLearner(
    model_regression=SeparateModel(
        RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001, random_state=123),
        RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    ),
    model_propensity=RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    cv=5,
)
dr_forest_sep.fit(y, D, W=W)
print("-----------------Random Forest in EconML for IRM (separate models)------------")
print(dr_forest_sep.summary(T=1))

# decision tree
dr_tree = LinearDRLearner(
    model_regression=DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    model_propensity=DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    cv=5,
)
dr_tree.fit(y, D, W=W)
print("-----------------Decision Tree in EconML for IRM------------")
print(dr_tree.summary(T=1))

# decision tree using seperate models for model_regression
dr_tree_sep = LinearDRLearner(
    model_regression=SeparateModel(
        DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001, random_state=123),
        DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    ),
    model_propensity=DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    cv=5,
)
dr_tree_sep.fit(y, D, W=W)
print("-----------------Decision Tree in EconML for IRM (separate models)------------")
print(dr_tree_sep.summary(T=1))

# gradient boosting
dr_gb = LinearDRLearner(
    model_regression=GradientBoostingRegressor(max_depth=2, n_iter_no_change=5, random_state=123),
    model_propensity=GradientBoostingClassifier(max_depth=2, n_iter_no_change=5, random_state=123),
    cv=5,
)
dr_gb.fit(y, D, W=W)
print("-----------------Gradient Boosting in EconML for IRM------------")
print(dr_gb.summary(T=1))

# gradient boosting using seperate models for model_regression
dr_gb_sep = LinearDRLearner(
    model_regression=SeparateModel(
        GradientBoostingRegressor(max_depth=2, n_iter_no_change=5, random_state=123),
        GradientBoostingRegressor(max_depth=2, n_iter_no_change=5, random_state=123),
    ),
    model_propensity=GradientBoostingClassifier(max_depth=2, n_iter_no_change=5, random_state=123),
    cv=5,
)
dr_gb_sep.fit(y, D, W=W)
print("-----------------Gradient Boosting in EconML for IRM (separate models)------------")
print(dr_gb_sep.summary(T=1))
# irm with double ml

# random forest
dml_irm_rf = dbml.DoubleMLIRM(
    dml_data,
    RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    n_folds=5,
)
dml_irm_rf.fit()
print("-----------------Random Forest in DoubleML for IRM------------")
print(dml_irm_rf.summary)

# decision tree

dml_irm_dt = dbml.DoubleMLIRM(
    dml_data,
    DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001, random_state=123),
    n_folds=5,
)
dml_irm_dt.fit()
print("-----------------Decision Tree in DoubleML for IRM------------")
print(dml_irm_dt.summary)
# gradient boosting

dml_irm_gb = dbml.DoubleMLIRM(
    dml_data,
    GradientBoostingRegressor(max_depth=2, n_iter_no_change=5, random_state=123),
    GradientBoostingClassifier(max_depth=2, n_iter_no_change=5, 
                               
                               random_state=123),
    n_folds=5,
)
dml_irm_gb.fit()
print("-----------------Gradient Boosting in DoubleML for IRM------------")
print(dml_irm_gb.summary)

----------------- Double Lasso in EconML for PLR------------
Coefficient Results:  X is None, please call intercept_inference to learn the constant!
                        CATE Intercept Results                        
               point_estimate  stderr  zstat pvalue ci_lower  ci_upper
----------------------------------------------------------------------
cate_intercept       8608.589 1426.197 6.036    0.0 5813.295 11403.884
----------------------------------------------------------------------

<sub>A linear parametric conditional average treatment effect (CATE) model was fitted:
$Y = \Theta(X)\cdot T + g(X, W) + \epsilon$
where for every outcome $i$ and treatment $j$ the CATE $\Theta_{ij}(X)$ has the form:
$\Theta_{ij}(X) = X' coef_{ij} + cate\_intercept_{ij}$
Coefficient Results table portrays the $coef_{ij}$ parameter vector for each outcome $i$ and treatment $j$. Intercept Results table portrays the $cate\_intercept_{ij}$ parameter.</sub>
-----------------Random Forest in Econ

Econml can work with most of the base learners (random forest, decision tree, boosted forest), as can doubleML. Both can work with any scikit-learn model in fact, so long as they implement fit() and predict() functions, according to the documentation. Theoretically, one could thus also build a custom class that implements the scikit api for stacking or semi crossfitting with choosing the best model, but neither library can work directly with stacking or perform the semi cross-fitting with the custom implementations we built as far as I could tell from the documentation of the packages. In theory one could write custom scikit-learn compatible interfaces for the custom implementations, but that would be a lot of work.  
  
In practice however, I found that double lasso did not work with the doubleML library. Upon closer examination, I found a section of the econml docs that explains that it specifically handles cases of models that do hyperparameter searches internally across folds. This is not the case for the doubleML library, which needs all folds to be treated strictly independently of each other under the hood and thus fails when LassoCV is used. This is a fundamental difference in the way the two libraries are built, and it means that doubleML cannot be used with double lasso as long as cross validation is used. One could however use a theoretically chosen penalty with lasso with scikit interface and then use that with doubleML.
   
As for the results (note that results may change slightly upon rerunning/exporting due to randomness. Please see output for full table, this is just for your convenience while grading. Estimate first, then standard error in parentheses, see above for other metrics): 
  
For PLR:  

double lasso previous: 9035	(1295)
double lasso econml: 8609 (1426)
double lasso doubleML:  n/a
  
random forest previous:  8905 (1357)  
random forest econml:  8603 (1333)   
random forest doubleML: 8523 (1346)  
  
decision tree previous: 9236 (1440)  
decision tree econml: 8772 (1449)  
decision tree doubleML: 8734 (1455)  
  
boosted forest previous: 8840 (1334)  
boosted forest econml: 9129 (1379)  
boosted forest doubleML: 8834 (1366)  

For IRM:  
random forest previous: 7699 (1159)  
random forest econml: 8023 (1121)  
random forest doubleML: 7805 (1155)  

decision tree previous: 7836 (1255)  
decision tree econml: 8352 (1250)  
decision tree doubleML: 7714 (1238) 

boosted forest previous: 8593 (1157)  
boosted forest econml: 8118 (1135)  
boosted forest doubleML: 8683 (1258)  

The results are broadly consistent across the different methods and libraries, including for the decision tree. Apart from the decision tree in IRM, the results are also consistent with the estimates from the custom methods. For EconML and Double ML, the decision tree in the IRM setting is more in line with the other models than for the custom implementation, where it differs significantly.

In [16]:
# d.) 

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.base import clone
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

class semisynth:
    
    def fit(self, X, D, y, transformer, random_state=None):
        """
        X, D, y: the real data
        transformer: any sklearn-compatible Transformer for pre-processing
        """
        self.X_ = X.copy()

        # Model for Y|D=0
        self.est0_ = make_pipeline(transformer,
                                   RandomForestRegressor(min_samples_leaf=20,
                                                         ccp_alpha=0.001,
                                                         random_state=random_state)
                                  ).fit(X[D==0], y[D==0])
        self.res0_ = y[D==0] - self.est0_.predict(X[D==0])
        # De-mean the residual distribution
        self.res0_ -= np.mean(self.res0_)

        # Model for Y|D=1
        self.est1_ = make_pipeline(transformer,
                                   RandomForestRegressor(min_samples_leaf=20,
                                                         ccp_alpha=0.001,
                                                         random_state=random_state)
                                  ).fit(X[D==1], y[D==1])
        self.res1_ = y[D==1] - self.est1_.predict(X[D==1])
        self.res1_ -= np.mean(self.res1_)

        # Model for D|X
        self.prop_ = make_pipeline(transformer,
                                   RandomForestClassifier(min_samples_leaf=20,
                                                          ccp_alpha=0.001,
                                                          random_state=random_state)
                                  ).fit(X, D)
        return self

    def generate_data(self, n):
        """
        Returns (X, D, Y, Y1, Y0):
          X, D, Y: the new sample
          Y1, Y0: potential outcomes for each row
        """
        # Resample X from the empirical distribution
        X = self.X_.iloc[np.random.choice(self.X_.shape[0], n, replace=True)]
        
        # Simulate D ~ Bernoulli(\hat{p}(X))
        pX = self.prop_.predict_proba(X)[:, 1]
        D = np.random.binomial(1, pX)

        # Construct Y0, Y1 by re-sampling from residual distribution
        y0 = self.est0_.predict(X) + self.res0_[np.random.choice(self.res0_.shape[0], n, replace=True)]
        y1 = self.est1_.predict(X) + self.res1_[np.random.choice(self.res1_.shape[0], n, replace=True)]
        
        # Observed Y
        y = y0*(1 - D) + y1*D
        return X, D, y, y1, y0
    
    def y_cef(self, X, D):
        """
        Returns the 'true' E[Y|X, D] in the semi-synthetic world
        = the random forest predictions from the original data
        """
        return self.est1_.predict(X)*D + self.est0_.predict(X)*(1 - D)
    
    def D_cef(self, X):
        """
        Returns the 'true' E[D|X] in the semi-synthetic world
        """
        return self.prop_.predict_proba(X)[:, 1]

    @property
    def true_ate(self):
        """
        The 'true' ATE in the semi-synthetic world, i.e. E[f1(X) - f0(X)]
        using the entire original X_ distribution.
        """
        return np.mean(self.est1_.predict(self.X_) - self.est0_.predict(self.X_))


def summary(
    point, stderr,
    yhat, Dhat,    # cross-fitted predictions for y and D
    resy, resD,    # residuals y-yhat, D-Dhat
    final_residual, # epsilon or drhat
    X, D, y,
    *,
    name,
    synth  # the semisynth object, so we can compare to the "true" functions
):
    true_ate = synth.true_ate
    covered = (point - 1.96*stderr <= true_ate <= point + 1.96*stderr)

    # We'll compute the "true" E[Y|D,X], E[D|X]
    y_cef_true = synth.y_cef(X, D)
    d_cef_true = synth.D_cef(X)

    return pd.DataFrame({
        'estimate':    [point],
        'stderr':      [stderr],
        'lower':       [point - 1.96*stderr],
        'upper':       [point + 1.96*stderr],
        'rmse y':      [np.sqrt(np.mean(resy**2))],  # RMSE vs. *observed* Y
        'rmse D':      [np.sqrt(np.mean(resD**2))],  # RMSE vs. *observed* D
        'accuracy D':  [np.mean(np.abs(resD) < 0.5)],# classification accuracy
        # New columns:
        'error':       [abs(point - true_ate)],    # how far from true
        'rmse E[y|D,X]':[np.sqrt(np.mean((yhat - y_cef_true)**2))],
        'rmse E[D|X]': [np.sqrt(np.mean((Dhat - d_cef_true)**2))],
        'covered':     [1 if covered else 0]       # did CI cover true ATE?
    }, index=[name])
    
    
from copy import deepcopy

synth = semisynth().fit(X, D, y, transformer, random_state=123)



def run_plr_methods(X_train, D_train, y_train, synth):
    """
    X_train, D_train, y_train come from synth.generate_data(...)
    We'll replicate your DML approach for partial linear model
    with multiple model combos and stack them in a table.
    """
    results_table = []

    # 1) Double Lasso with cross-fitting
    # (a) specify pipelines
    lassoy_ = deepcopy(lassoy)
    lassod_ = deepcopy(lassod)
    # (b) run
    point, stderr, yhat, Dhat, resy, resD, eps = dml(X_train, D_train, y_train, lassoy_, lassod_, nfolds=5)
    # (c) summary
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, eps,
                  X_train, D_train, y_train, name='double lasso', synth=synth)
    results_table.append(df_)

    # 2) lasso / logistic
    lassoy_ = deepcopy(lassoy)
    lgrd_   = deepcopy(lgrd)
    point, stderr, yhat, Dhat, resy, resD, eps = dml(X_train, D_train, y_train,
                                                     lassoy_, lgrd_, nfolds=5, classifier=True)
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, eps,
                  X_train, D_train, y_train, name='lasso/logistic', synth=synth)
    results_table.append(df_)

    # 3) Random Forest
    rfy_ = deepcopy(rfy)
    rfd_ = deepcopy(rfd)
    point, stderr, yhat, Dhat, resy, resD, eps = dml(X_train, D_train, y_train,
                                                     rfy_, rfd_, nfolds=5, classifier=True)
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, eps,
                  X_train, D_train, y_train, name='random forest', synth=synth)
    results_table.append(df_)

    # 4) Decision Tree
    dtry_ = deepcopy(dtry)
    dtrd_ = deepcopy(dtrd)
    point, stderr, yhat, Dhat, resy, resD, eps = dml(X_train, D_train, y_train,
                                                     dtry_, dtrd_, nfolds=5, classifier=True)
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, eps,
                  X_train, D_train, y_train, name='decision tree', synth=synth)
    results_table.append(df_)

    # 5) Boosted trees
    gbfy_ = deepcopy(gbfy)
    gbfd_ = deepcopy(gbfd)
    point, stderr, yhat, Dhat, resy, resD, eps = dml(X_train, D_train, y_train,
                                                     gbfy_, gbfd_, nfolds=5, classifier=True)
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, eps,
                  X_train, D_train, y_train, name='boosted forest', synth=synth)
    results_table.append(df_)

    # 6) automl (semi-cfit)
    # Similarly for stacking (semi-cfit).
    flamly_ = make_pipeline(transformer, AutoML(time_budget=50,
                                                task='regression',
                                                early_stop=True,
                                                eval_method='cv',
                                                n_splits=3,
                                                metric='r2',
                                                verbose=0))
    flamld_ = make_pipeline(transformer, AutoML(time_budget=50,
                                                task='classification',
                                                early_stop=True,
                                                eval_method='cv',
                                                n_splits=3,
                                                metric='r2',
                                                verbose=0))
    # Fit Y, D on entire X_train
    flamly_.fit(X_train, y_train)
    besty_model = flamly_[-1].best_model_for_estimator(flamly_[-1].best_estimator)
    besty = make_pipeline(transformer, clone(besty_model))

    flamld_.fit(X_train, D_train)
    bestd_model = flamld_[-1].best_model_for_estimator(flamld_[-1].best_estimator)
    bestd = make_pipeline(transformer, clone(bestd_model))

    point, stderr, yhat, Dhat, resy, resD, eps = dml(X_train, D_train, y_train,
                                                     besty, bestd, nfolds=5, classifier=True)
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, eps,
                  X_train, D_train, y_train, name='automl (semi-cfit)', synth=synth)
    results_table.append(df_)

    # 7) stacking (semi-cfit)
    point, stderr, yhat, Dhat, resy, resD, eps = dml_dirty(
        X_train, D_train, y_train,
        [lassoy, rfy, dtry, gbfy],
        [lgrd, rfd, dtrd, gbfd],
        nfolds=5, classifier=True
    )
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, eps,
                  X_train, D_train, y_train, name='stacked (semi-cfit)', synth=synth)
    results_table.append(df_)
    
    #8 select best
    point, stderr, yhat, Dhat, resy, resD, eps = dml_select_best(
        X_train, D_train, y_train,
        [lassoy, rfy, dtry, gbfy],
        [lgrd, rfd, dtrd, gbfd],
        nfolds=5, classifier=True
    )
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, eps,
                    X_train, D_train, y_train, name='select-best (semi-cfit)', synth=synth)
    results_table.append(df_)

    return pd.concat(results_table)


def run_irm_methods(X_train, D_train, y_train, synth):
    """
    X_train, D_train, y_train from synth.generate_data(...)
    We'll replicate your IRM approach with multiple model combos and stack in a table.
    """
    results_table = []

    # 1) lasso-lasso, logistic repeated seeds + median aggregator
    # We'll do a single run for demonstration:
    lassoy_ = deepcopy(lassoytest)  # or define a pipeline as in the notebook
    lgrd_   = deepcopy(lgrdtest)

    point, stderr, yhat, Dhat, resy, resD, drhat = dr(X_train, D_train, y_train,
                                                      lassoy_, lassoy_,
                                                      lgrd_, nfolds=5)
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, drhat,
                  X_train, D_train, y_train, name='lasso/logistic', synth=synth)
    results_table.append(df_)

    # 2) random forest
    rfy_ = deepcopy(rfy)
    rfd_ = deepcopy(rfd)
    point, stderr, yhat, Dhat, resy, resD, drhat = dr(X_train, D_train, y_train,
                                                      rfy_, rfy_, rfd_, nfolds=5)
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, drhat,
                  X_train, D_train, y_train, name='random forest', synth=synth)
    results_table.append(df_)

    # 3) decision tree
    dtry_ = deepcopy(dtry)
    dtrd_ = deepcopy(dtrd)
    point, stderr, yhat, Dhat, resy, resD, drhat = dr(X_train, D_train, y_train,
                                                      dtry_, dtry_, dtrd_, nfolds=5)
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, drhat,
                  X_train, D_train, y_train, name='decision tree', synth=synth)
    results_table.append(df_)

    # 4) boosted forest
    gbfy_ = deepcopy(gbfy)
    gbfd_ = deepcopy(gbfd)
    point, stderr, yhat, Dhat, resy, resD, drhat = dr(X_train, D_train, y_train,
                                                      gbfy_, gbfy_, gbfd_, nfolds=5)
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, drhat,
                  X_train, D_train, y_train, name='boosted forest', synth=synth)
    results_table.append(df_)

    # 5) automl
    flamly0_ = make_pipeline(transformer, AutoML(time_budget=30, task='regression', early_stop=True,
                                                eval_method='cv', n_splits=3, metric='r2', verbose=0))
    flamly1_ = make_pipeline(transformer, AutoML(time_budget=30, task='regression', early_stop=True,
                                                eval_method='cv', n_splits=3, metric='r2', verbose=0))
    flamld_  = make_pipeline(transformer, AutoML(time_budget=30, task='classification', early_stop=True,
                                                eval_method='cv', n_splits=3, metric='r2', verbose=0))
    # Fit for Y|D=0, Y|D=1
    flamly0_.fit(X_train[D_train == 0], y_train[D_train == 0])
    besty0_model = flamly0_[-1].best_model_for_estimator(flamly0_[-1].best_estimator)
    besty0 = make_pipeline(transformer, clone(besty0_model))

    flamly1_.fit(X_train[D_train == 1], y_train[D_train == 1])
    besty1_model = flamly1_[-1].best_model_for_estimator(flamly1_[-1].best_estimator)
    besty1 = make_pipeline(transformer, clone(besty1_model))

    flamld_.fit(X_train, D_train)
    bestd_model = flamld_[-1].best_model_for_estimator(flamld_[-1].best_estimator)
    bestd = make_pipeline(transformer, clone(bestd_model))

    point, stderr, yhat, Dhat, resy, resD, drhat = dr(X_train, D_train, y_train,
                                                      besty0, besty1, bestd,
                                                      nfolds=5)
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, drhat,
                  X_train, D_train, y_train, name='automl (semi-cfit)', synth=synth)
    results_table.append(df_)

    # 6) stacking (semi-cfit):
    lassoy_ = deepcopy(lassoy)
    rfy_ = deepcopy(rfy)
    dtry_ = deepcopy(dtry)
    gbfy_ = deepcopy(gbfy)

    lgrd_ = deepcopy(lgrd)
    rfd_  = deepcopy(rfd)
    dtrd_ = deepcopy(dtrd)
    gbfd_ = deepcopy(gbfd)

    point, stderr, yhat, Dhat, resy, resD, drhat = dr_dirty(
        X_train, D_train, y_train,
        [lassoy_, rfy_, dtry_, gbfy_],
        [lassoy_, rfy_, dtry_, gbfy_],
        [lgrd_, rfd_, dtrd_, gbfd_],
        nfolds=5
    )
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, drhat,
                  X_train, D_train, y_train, name='stacked (semi-cfit)', synth=synth)
    results_table.append(df_)
    
    # 7) select best
    
    point, stderr, yhat, Dhat, resy, resD, drhat = dr_select_best(
        X_train, D_train, y_train,
        [lassoy, rfy, dtry, gbfy],
        [lassoy, rfy, dtry, gbfy],
        [lgrd, rfd, dtrd, gbfd],
        nfolds=5
    )
    df_ = summary(point, stderr, yhat, Dhat, resy, resD, drhat,
                    X_train, D_train, y_train, name='select-best (semi-cfit)', synth=synth)
    results_table.append(df_)

    return pd.concat(results_table)


for n in [1000, 10000, 50000]:
    print(f"\n=== Semi-Synthetic Data with n={n} ===")
    print("True ATE in the semi-synthetic world: ", synth.true_ate)
    X_synth, D_synth, y_synth, y1_synth, y0_synth = synth.generate_data(n)


    print(f"** PLR Results n = {n} **")
    print("True ATE in the semi-synthetic world: ", synth.true_ate)
    table_plr = run_plr_methods(X_synth, D_synth, y_synth, synth)
    print(table_plr)

    print(f"** IRM Results n = {n}**")
    table_irm = run_irm_methods(X_synth, D_synth, y_synth, synth)
    print(table_irm)


=== Semi-Synthetic Data with n=1000 ===
True ATE in the semi-synthetic world:  7448.65797745209
** PLR Results n = 1000 **
True ATE in the semi-synthetic world:  7448.65797745209
                            estimate       stderr        lower         upper  \
double lasso             4473.600355  3541.878591 -2468.481682  11415.682393   
lasso/logistic           5057.826381  3454.633691 -1713.255652  11828.908415   
random forest            6414.514167  3524.069645  -492.662337  13321.690672   
decision tree            2139.271255  3316.734745 -4361.528845   8640.071355   
boosted forest           3788.528551  3949.094321 -3951.696319  11528.753421   
automl (semi-cfit)       5084.189070  3397.308487 -1574.535564  11742.913704   
stacked (semi-cfit)      5085.361916  3473.896692 -1723.475600  11894.199432   
select-best (semi-cfit)  4810.734877  3625.171173 -2294.600623  11916.070377   

                               rmse y    rmse D  accuracy D        error  \
double lasso           

For n = 1000:    
In the PLR setting, random forest outperforms all other methods in terms of the estimate. 
In the IRM setting, the random forest outperforms all other methods in terms of the estimate.   
So neither automl nor stacking perform as well as the best model alone.  

For n = 10000:  

In the PLR setting, the decision tree performs best in terms of the estimate.     
In the IRM setting, lasso / logistic performs best in terms of the estimate.   
So neither stacking nor automl perform as well as the best model alone.   
  
For n = 50000:  
  
In the PLR setting, Decision tree performs best in terms of the estimate.   
In the IRM setting, Decision tree performs best in terms of the estimate.  
Neither automl nor stacking perform as well as the best model alone.  

  
Please note that running the code repeatedly showed that the results can vary due to randomness. The results above are from a single run and I saw other models perform best in some cases in other runs.  